In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)['keywords']

0    [{"id": 1463, "name": "culture clash"}, {"id":...
Name: keywords, dtype: object

In [4]:
movies = movies.merge(credits,on="title")

In [5]:
movies = movies[['genres','id','keywords','title','overview','cast','crew']]

In [6]:
movies.isnull().sum()

genres      0
id          0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [7]:

movies = movies.dropna()

In [8]:
movies.duplicated().sum()

0

In [9]:
movies.iloc[0].genres


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [10]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])   
    return L

In [11]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [12]:
def convertcast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
def director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
            

In [13]:
movies['cast'] = movies['cast'].apply(convertcast)
movies['crew'] = movies['crew'].apply(director)

In [14]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [15]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movies['tag'] = movies['genres'] + movies['keywords'] + movies['overview'] + movies['cast'] + movies['crew']
movies_df = movies[['id','title','tag']]

In [17]:
movies_df['tag'] = movies_df['tag'].apply(lambda x:" ".join(x))


C:\Users\Divyansh\AppData\Local\Temp\ipykernel_21860\1628052150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['tag'] = movies_df['tag'].apply(lambda x:" ".join(x))


In [18]:
movies_df['tag'] = movies_df['tag'].apply(lambda x:x.lower())

C:\Users\Divyansh\AppData\Local\Temp\ipykernel_21860\3369079983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['tag'] = movies_df['tag'].apply(lambda x:x.lower())


In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [20]:
def stemming(text):
    L = []
    
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)

In [21]:
movies_df['tag'] = movies_df['tag'].apply(stemming)

C:\Users\Divyansh\AppData\Local\Temp\ipykernel_21860\4141830777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['tag'] = movies_df['tag'].apply(stemming)


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',max_features=5000)

In [23]:
arr = cv.fit_transform(movies_df['tag']).toarray()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(arr)

In [25]:
def rec(movie):
    movie_index = movies_df[movies_df["title"] == movie].index[0]
    movies = sorted(list(enumerate(similarity[movie_index])),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies:
        print(movies_df.iloc[i[0]]['title'])

In [26]:
rec('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [27]:
import pickle

In [28]:
pickle.dump(movies_df,open('movies.pkl','wb'))

In [29]:
pickle.dump(similarity,open('similarity.pkl','wb'))